In [6]:
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm

#from urllib import urlretrieve
#import cPickle as pickle
import os
#import gzip

import numpy as np
import theano

import lasagne
from lasagne import layers
from lasagne.updates import adam

#from nolearn.lasagne import NeuralNet
#from nolearn.lasagne import visualize

#from ConvTranspose import Conv3DLayerTransposed

#from sklearn.metrics import classification_report
#from sklearn.metrics import confusion_matrix

In [7]:
import theano.tensor as T

from lasagne import init
from lasagne import nonlinearities
from lasagne.utils import as_tuple
from lasagne.theano_extensions import conv

from lasagne.layers.base import Layer

def conv_output_length(input_length, filter_size, stride, pad=0):
    if input_length is None:
        return None
    if pad == 'valid':
        output_length = input_length - filter_size + 1
    elif pad == 'full':
        output_length = input_length + filter_size - 1
    elif pad == 'same':
        output_length = input_length
    elif isinstance(pad, int):
        output_length = input_length + 2 * pad - filter_size + 1
    else:
        raise ValueError('Invalid pad: {0}'.format(pad))

    # This is the integer arithmetic equivalent to
    # np.ceil(output_length / stride)
    output_length = (output_length + stride - 1) // stride

    return output_length


def conv_input_length(output_length, filter_size, stride, pad=0):
      
    if output_length is None:
        return None
    if pad == 'valid':
        pad = 0
    elif pad == 'full':
        pad = filter_size - 1
    elif pad == 'same':
        pad = filter_size // 2
    if not isinstance(pad, int):
        raise ValueError('Invalid pad: {0}'.format(pad))
    return (output_length - 1) * stride - 2 * pad + filter_size


class BaseConvLayer(Layer):
    def __init__(self, incoming, num_filters, filter_size, stride=1, pad=0,
                 untie_biases=False,
                 W=init.GlorotUniform(), b=init.Constant(0.),
                 nonlinearity=nonlinearities.rectify, flip_filters=True,
                 n=None, **kwargs):
        super(BaseConvLayer, self).__init__(incoming, **kwargs)
        if nonlinearity is None:
            self.nonlinearity = nonlinearities.identity
        else:
            self.nonlinearity = nonlinearity

        if n is None:
            n = len(self.input_shape) - 2
        elif n != len(self.input_shape) - 2:
            raise ValueError("Tried to create a %dD convolution layer with "
                             "input shape %r. Expected %d input dimensions "
                             "(batchsize, channels, %d spatial dimensions)." %
                             (n, self.input_shape, n+2, n))
        self.n = n
        self.num_filters = num_filters
        self.filter_size = as_tuple(filter_size, n, int)
        self.flip_filters = flip_filters
        self.stride = as_tuple(stride, n, int)
        self.untie_biases = untie_biases

        if pad == 'same':
            if any(s % 2 == 0 for s in self.filter_size):
                raise NotImplementedError(
                    '`same` padding requires odd filter size.')
        if pad == 'valid':
            self.pad = as_tuple(0, n)
        elif pad in ('full', 'same'):
            self.pad = pad
        else:
            self.pad = as_tuple(pad, n, int)

        self.W = self.add_param(W, self.get_W_shape(), name="W")
        if b is None:
            self.b = None
        else:
            if self.untie_biases:
                biases_shape = (num_filters,) + self.output_shape[2:]
            else:
                biases_shape = (num_filters,)
            self.b = self.add_param(b, biases_shape, name="b",
                                    regularizable=False)

    def get_W_shape(self):
        """Get the shape of the weight matrix `W`.
        Returns
        -------
        tuple of int
            The shape of the weight matrix.
        """
        num_input_channels = self.input_shape[1]
        return (self.num_filters, num_input_channels) + self.filter_size

    def get_output_shape_for(self, input_shape):
        pad = self.pad if isinstance(self.pad, tuple) else (self.pad,) * self.n
        batchsize = input_shape[0]
        return ((batchsize, self.num_filters) +
                tuple(conv_output_length(input, filter, stride, p)
                      for input, filter, stride, p
                      in zip(input_shape[2:], self.filter_size,
                             self.stride, pad)))

    def get_output_for(self, input, **kwargs):
        conved = self.convolve(input, **kwargs)

        if self.b is None:
            activation = conved
        elif self.untie_biases:
            activation = conved + T.shape_padleft(self.b, 1)
        else:
            activation = conved + self.b.dimshuffle(('x', 0) + ('x',) * self.n)

        return self.nonlinearity(activation)

    def convolve(self, input, **kwargs):
        """
        Symbolically convolves `input` with ``self.W``, producing an output of
        shape ``self.output_shape``. To be implemented by subclasses.
        Parameters
        ----------
        input : Theano tensor
            The input minibatch to convolve
        **kwargs
            Any additional keyword arguments from :meth:`get_output_for`
        Returns
        -------
        Theano tensor
            `input` convolved according to the configuration of this layer,
            without any bias or nonlinearity applied.
        """
        raise NotImplementedError("BaseConvLayer does not implement the "
                                  "convolve() method. You will want to "
                                  "use a subclass such as Conv2DLayer.")

class Conv3DLayerTransposed(BaseConvLayer):

    #pad spremenim v crop, flip_filters nastavim na True, spremenim super()
    def __init__(self, incoming, num_filters, filter_size, stride=(1, 1, 1),
                 crop=0, untie_biases=False,
                 W=init.GlorotUniform(), b=init.Constant(0.),
                 nonlinearity=nonlinearities.rectify, flip_filters=False,
                 convolution=T.nnet.ConvTransp3D, output_size=None, **kwargs):
        super(Conv3DLayerTransposed, self).__init__(incoming, num_filters, filter_size,
                                          stride, crop, untie_biases, W, b,
                                          nonlinearity, flip_filters, n=3,
                                          **kwargs)
        self.crop = self.pad
        del self.pad
        self.convolution = convolution
        self.output_size = output_size
        
    def get_W_shape(self):
        num_input_channels = self.input_shape[1]
        # first two sizes are swapped compared to a forward convolution
        return (num_input_channels, self.num_filters) + self.filter_size
    
    def get_output_shape_for(self, input_shape):
        #if self.output_size is not None:
         #   size = self.output_size
        #     if isinstance(self.output_size, T.Variable):
        #        size = (None, None)
        #    return input_shape[0], self.num_filters, size[0], size[1]

        # If self.output_size is not specified, return the smallest shape
        # when called from the constructor, self.crop is still called self.pad:
        crop = getattr(self, 'crop', getattr(self, 'pad', None))
        crop = crop if isinstance(crop, tuple) else (crop,) * self.n
        batchsize = input_shape[0]
        return ((batchsize, self.num_filters) +
                tuple(conv_input_length(input, filter, stride, p)
                      for input, filter, stride, p
                      in zip(input_shape[2:], self.filter_size,
                             self.stride, crop)))

    #pad v crop, filter_flip nastavim na not 
    #def convolve(self, input, **kwargs):
    #    border_mode = 'half' if self.crop == 'same' else self.crop
    #    conved = self.convolution(input, self.W, self.input_shape, self.get_W_shape(),
    #                              subsample=self.stride,
    #                              border_mode=border_mode,
    #                              filter_flip=not self.flip_filters)
    #    return conved
    
    def convolve(self, input, **kwargs):
        border_mode = 'half' if self.crop == 'same' else self.crop
        op = T.nnet.abstract_conv.AbstractConv3d_gradInputs(
            imshp=self.output_shape,
            kshp=self.get_W_shape(),
            subsample=self.stride, border_mode=border_mode,
            filter_flip=not self.flip_filters)
        output_size = self.output_shape[2:]
        if isinstance(self.output_size, T.Variable):
            output_size = self.output_size
        elif any(s is None for s in output_size):
            output_size = self.get_output_shape_for(input.shape)[2:]
        conved = op(self.W, input, output_size)
        return conved

In [8]:
def contracting_block(layer):
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(2, 2, 2), stride=2, pad=0, W=lasagne.init.Normal())
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    tmp_layer = layer
    #ugotovi primeren batch norm layer
    layer = layers.BatchNormLayer(layer)
    #ugotovi primeren prelu layer
    layer = layers.prelu(layer)
    #ugotovi num_filters, padding
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=1, pad=1, W=lasagne.init.Normal())
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    layer = layers.ElemwiseSumLayer([layer, tmp_layer])
    layer = layers.batch_norm(layer)
    layer = layers.prelu(layer)
    return layer
    

In [9]:
def start(input):
    layer = layers.Conv3DLayer(input, num_filters=1, filter_size=(3, 3, 3), stride=1, pad=1, W=lasagne.init.Normal())
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    #print(theano.tensor.shape(lasagne.layers.get_output(layer)))
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [10]:
def expanding_block(layer, tmp):
    # dodaj feature merging block
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=0, W=lasagne.init.Normal())
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    # kako deluje deconvolutional layer --> DEKONVOLUCIJA
    #layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=0.5)
    layer = Conv3DLayerTransposed(layer, num_filters=1, filter_size=(2, 2, 2), stride=2, crop=0, W=lasagne.init.Normal())
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    layer = layers.ConcatLayer([layer, tmp])
    layer = layers.Conv3DLayer(layer, num_filters=1, filter_size=(3, 3, 3), stride=1, pad=1, W=lasagne.init.Normal())
    layer = layers.BatchNormLayer(layer)
    layer = layers.prelu(layer)
    return layer

In [11]:
def binary_jaccard_index(predictions,targets):
    intersection = theano.tensor.minimum(predictions, targets)
    union = theano.tensor.maximum(predictions, targets)
    axes = tuple(range(1,5))
    return intersection.sum(axis=axes) / union.sum(axis=axes)

In [12]:
#prikaze eno 3d sliko
def display_numpy(picture):
    fig = plt.figure()
    for num,slice in enumerate(picture):
        y = fig.add_subplot(4,6,num+1)
        y.imshow(slice, cmap='gray')
    plt.show()

In [13]:
#load the data

data = np.load('data-160-160-24.npy')
seg_data = np.load('segdata-160-160-24.npy')

data = np.expand_dims(data, axis=1)
seg_data = np.expand_dims(seg_data, axis=1)

print(data.shape)
print(seg_data.shape)

train_data = data[:-2]
validation_data = data[-2:]
train_seg = seg_data[:-2]
validation_seg = seg_data[-2:]

#display_numpy(train_data[1][0])



(10L, 1L, 24L, 160L, 160L)
(10L, 1L, 24L, 160L, 160L)


In [ ]:
input_var = theano.tensor.tensor5()
target_var = theano.tensor.tensor5()


layer = lasagne.layers.InputLayer((None, 1, 24, 160, 160), input_var)
layer = start(layer)
tmp1 = layer
layer = contracting_block(layer)
tmp2 = layer
layer = contracting_block(layer)
tmp3 = layer
layer = contracting_block(layer)
layer = expanding_block(layer, tmp3)
#tmp4 = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=0)
layer = expanding_block(layer, tmp2)
#tmp5 = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=0)
layer = expanding_block(layer, tmp1)
#tmp6 = layers.Conv3DLayer(layer, num_filters=1, filter_size=(1, 1, 1), stride=1, pad=0)

prediction = lasagne.layers.get_output(layer)
#uporabiti pravi loss metric
loss = lasagne.objectives.squared_error(prediction, target_var)
loss = loss.mean() + 1e-4 * lasagne.regularization.regularize_network_params(layer, lasagne.regularization.l2)

#print(loss.eval())

# create parameter update expressions
params = lasagne.layers.get_all_params(layer, trainable=True)
updates = lasagne.updates.adam(loss, params, learning_rate=0.001)

# compile training function that updates parameters and returns training loss
train_fn = theano.function([input_var, target_var], [loss, prediction], updates=updates)

# train network (assuming you've got some training data in numpy arrays)
for epoch in range(10):
    #visualize.plot_conv_weights(layer)
    #print(lasagne.utils.floatX(prediction).shape)
    loss = train_fn(train_data, train_seg)
    #print(loss[1].shape)
    #print(type(loss[1]))
    #display_numpy(loss[1][0][0]) #tole odkomentiraj za slikce :)
    #theano.printing.pprint(prediction[2][2]) 
    print("Epoch %d: Loss %g" % (epoch + 1, loss[0]))

In [94]:
x = 2
print(eval('x'))

2
